In [25]:
!pip install -q langchain langchain-community langchain-text-splitters
!pip install -q langchain-anthropic langchain-core
!pip install -q pymupdf faiss-cpu sentence-transformers
!pip install -q huggingface-hub transformers torch python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.3 MB/s eta 0:00:00


In [26]:
import subprocess
import sys
import os
import numpy as np
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from urllib.request import urlretrieve
import warnings
from pathlib import Path
from typing import List, Dict
warnings.filterwarnings('ignore')

In [67]:
PDF_FILES = [
    "/content/MachineLearning-supervised.pdf",
    "/content/MLBasics-Unsupervised.pdf"
]
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50

In [68]:
chunks = load_and_split_multiple_pdfs(PDF_FILES)

📚 Loading Multiple PDFs
✅ Loaded 84 pages
✅ Split into 87 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineer - July 2024
Machine Learning Basics
Your AI Starter Kit
✅ Loaded 109 pages
✅ Split into 114 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineers - August 2024
Unsupervised Learning
Machine Learning Basics - Continue

📊 SUMMARY
Total PDFs processed: 2
Total chunks created: 201


In [69]:
def load_and_split_single_pdf(pdf_path):
    """Load and split a single PDF"""
    loader = PyMuPDFLoader(pdf_path)
    docs = loader.load()
    print(f"✅ Loaded {len(docs)} pages")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    chunks = text_splitter.split_documents(docs)

    print(f"✅ Split into {len(chunks)} chunks")
    if chunks:
        print("🧩 Sample chunk:\n", chunks[0].page_content[:300])
    else:
        print("⚠️ No text extracted — check if the PDF contains selectable text.")

    return chunks


# Run
if __name__ == "__main__":
    chunks = load_and_split_multiple_pdfs(PDF_FILES)

📚 Loading Multiple PDFs
✅ Loaded 84 pages
✅ Split into 87 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineer - July 2024
Machine Learning Basics
Your AI Starter Kit
✅ Loaded 109 pages
✅ Split into 114 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineers - August 2024
Unsupervised Learning
Machine Learning Basics - Continue

📊 SUMMARY
Total PDFs processed: 2
Total chunks created: 201


In [70]:

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load the PDF
loader = PyMuPDFLoader(PDF_FILES[0])
docs_before_split = loader.load()
print(f"Loaded {len(docs_before_split)} documents")

# Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

docs_after_split = text_splitter.split_documents(docs_before_split)
print(f"Split into {len(docs_after_split)} chunks")
print("Sample chunk:\n", docs_after_split[0].page_content[:300])


Loaded 84 documents
Split into 95 chunks
Sample chunk:
 Lara Wehbe - TheAIEngineer - July 2024
Machine Learning Basics
Your AI Starter Kit


In [71]:
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

def load_and_split_documents(path):
    """
    Load PDFs (single file or folder) and split them into chunks
    """
    # Detect if it's a file or directory
    if os.path.isfile(path) and path.endswith(".pdf"):
        loader = PyMuPDFLoader(path)
    elif os.path.isdir(path):
        loader = PyPDFDirectoryLoader(path)
    else:
        raise ValueError("Invalid path: provide a PDF file or folder of PDFs.")

    docs_before_split = loader.load()
    print(f"Loaded {len(docs_before_split)} documents")

    # Split documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
    docs_after_split = text_splitter.split_documents(docs_before_split)

    print(f"Split into {len(docs_after_split)} chunks")
    if docs_after_split:
        print("Sample chunk:\n", docs_after_split[0].page_content[:300])
    else:
        print("No text found. Check if PDFs contain readable text.")

    return docs_after_split

# Run example
if __name__ == "__main__":
    chunks = load_and_split_documents(PDF_FILES[0])

Loaded 84 documents
Split into 95 chunks
Sample chunk:
 Lara Wehbe - TheAIEngineer - July 2024
Machine Learning Basics
Your AI Starter Kit


In [72]:
def create_vector_store(docs):
    """Create FAISS vector store with embeddings"""
    # Initialize embeddings
    embeddings = HuggingFaceBgeEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'},
        encode_kwargs={'normalize_embeddings': True}
    )

    # Create vector store
    vector_db = FAISS.from_documents(docs, embeddings)

    return vector_db


In [73]:

def setup_llm():
    """Setup HuggingFace Pipeline LLM"""
    # Use HuggingFacePipeline instead of HuggingFaceHub
    llm = HuggingFacePipeline.from_model_id(
        model_id="google/flan-t5-base",
        task="text2text-generation",
        model_kwargs={
            "temperature": 0.2,
            "max_length": 128,
            "do_sample": True
        },
        pipeline_kwargs={
            "max_new_tokens": 128
        }
    )

    return llm

In [80]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def create_rag_chain(vector_db, llm):
    """Create RAG chain using modern LCEL (no deprecated RetrievalQA)"""

    # Create retriever
    retriever = vector_db.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )

    # Define custom prompt template
    prompt_template = """Use the following pieces of context to answer the question at the end. Please follow the following rules:
1. If you don't know the answer, don't try to make up an answer. Just say "I can't find the final answer but you may want to check the following links".
2. If you find the answer, write the answer in a concise way with five sentences maximum.

Context:
{context}

Question:{question}

Helpful Answer:"""

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # Format retrieved documents
    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    # Create chain using LCEL (modern approach)
    rag_chain = (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough()
        }
        | PROMPT
        | llm
        | StrOutputParser()
    )

    return rag_chain, retriever

In [81]:

!pip install faiss-cpu


In [82]:

def main():
    print("\n" + "="*60)
    print("🚀 RAG SYSTEM STARTING")
    print("="*60 + "\n")

    # Load documents
    chunks = load_and_split_multiple_pdfs(PDF_FILES)

    if not chunks:
        print("❌ No documents loaded!")
        return

    # Create vector store
    vector_db = create_vector_store(chunks)

    # Setup LLM
    llm = setup_llm()

    # Create chain
    chain, retriever = create_rag_chain(vector_db, llm)

    # Test query
    print("\n" + "="*60)
    print("🧪 TEST QUERY")
    print("="*60)

    test_query = "What is supervised learning?"
    print(f"Query: {test_query}\n")

    print("Searching...")
    answer = chain.invoke(test_query)
    print(f"Answer: {answer}\n")

    # Interactive mode
    print("="*60)
    print("💬 INTERACTIVE MODE")
    print("="*60)
    print("Type 'quit' to exit\n")

    while True:
        question = input("Your question: ").strip()

        if question.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break

        if not question:
            continue

        print("\n⏳ Thinking...")
        answer = chain.invoke(question)
        print(f"Answer: {answer}\n")

# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    main()


🚀 RAG SYSTEM STARTING

📚 Loading Multiple PDFs
✅ Loaded 84 pages
✅ Split into 87 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineer - July 2024
Machine Learning Basics
Your AI Starter Kit
✅ Loaded 109 pages
✅ Split into 114 chunks
🧩 Sample chunk:
 Lara Wehbe - TheAIEngineers - August 2024
Unsupervised Learning
Machine Learning Basics - Continue

📊 SUMMARY
Total PDFs processed: 2
Total chunks created: 201


Device set to use cuda:0



🧪 TEST QUERY
Query: What is supervised learning?

Searching...
Answer: a category of Machine Learning that uses labeled datasets to train algorithms to predict outcomes and recognize patterns.

💬 INTERACTIVE MODE
Type 'quit' to exit

Your question: What is supervised learning

⏳ Thinking...
Answer: a category of Machine Learning that uses labeled datasets to train algorithms to predict outcomes and recognize patterns.

Your question: what is machine learning

⏳ Thinking...
Answer: ML is a subset of AI that focuses on the development of algorithms that allow computers to learn from and make predictions based on data.

Your question: quit
👋 Goodbye!
